In [5]:
%pip install tf-keras

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import chromadb


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
#input_file = "maharashtra_data.csv"
#Docs = pd.read_csv(input_file)
loader = PyPDFLoader("i3325e.pdf")
Docs = loader.load()

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000,chunk_overlap = 0)
chunks = text_splitter.split_documents(Docs)



In [8]:
print(type(chunks))
print(type(chunks[0]))
text_chunk = [chunk.page_content for chunk in chunks]
print(type(text_chunk[0]))

<class 'list'>
<class 'langchain_core.documents.base.Document'>
<class 'str'>


In [9]:
res = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = res.encode(text_chunk)

In [10]:
print(embeddings[600])
len(embeddings)

[ 5.67397773e-02 -9.08005759e-02  6.88941032e-02  4.00673263e-02
  9.30374563e-02 -3.07192951e-02 -1.75422821e-02 -5.53416200e-02
  1.21342838e-02 -2.31432449e-02  2.60299351e-02 -4.38701995e-02
 -5.71108609e-02 -5.10607772e-02 -6.90211877e-02 -9.98858735e-03
  2.95174699e-02 -1.89362112e-02 -6.86524287e-02 -7.34587908e-02
  9.49105173e-02  3.08655333e-02  1.54499635e-02  3.63667458e-02
 -1.99430380e-02 -6.43553957e-02 -2.26059258e-02 -1.23603772e-02
 -3.12908031e-02 -1.09654199e-02 -3.17397937e-02 -1.06219361e-02
  5.48833609e-02  2.17009615e-02 -1.69888530e-02  1.07967304e-02
  1.34988636e-01 -7.91748147e-03  3.13588942e-04 -9.86388605e-03
 -1.09062651e-02 -7.86580220e-02  4.13224995e-02 -4.58408222e-02
 -6.32772222e-02  3.89286131e-02 -1.37849199e-02 -3.89040820e-02
 -1.21909499e-01 -1.93848112e-03  7.06829652e-02  4.04629782e-02
 -3.51075493e-02 -1.36630051e-02  5.86281903e-02 -7.74825588e-02
 -3.45220342e-02 -5.38882501e-02 -3.22068594e-02  9.51288734e-03
 -1.56987943e-02 -5.00379

2094

In [12]:
client = chromadb.PersistentClient(path='database')
collection  =client.get_or_create_collection('collection')

ids = [f"doc_{i}" for i in range(len(embeddings))]

# Define the maximum batch size
max_batch_size = 166

# Split the data into smaller batches
for i in range(0, len(embeddings), max_batch_size):
	batch_ids = ids[i:i + max_batch_size]
	batch_documents = text_chunk[i:i + max_batch_size]
	batch_embeddings = embeddings[i:i + max_batch_size]
	collection.add(ids=batch_ids, documents=batch_documents, embeddings=batch_embeddings)

In [9]:
collection.peek()
collection.count()

2094

In [13]:
print(chunks[455])
len(chunks)

page_content='Restoring degraded soils for climate change resilience 123
   Adaptive management capacity  123
4.5 Conclusions  124
Notes      124
Acronyms      125
References      126
Additional Resources   131
Annex       132
A.4.1  Soil management for sustained crop productivity 
and climate change adaptation and mitigation  132
List of Figures
Figure 4.1 Definition of soil. Soils contribute to a range of vital ecosystem services and functions  109
Figure 4.2 Provision of ecosystem services and functions by soils.  110
Figure 4.3 Organic matter turnover  110
Figure 4.4 Estimates of global soil organic carbon (t/ha of carbon) 
from amended Harmonized World Soil Database  114
Figure 4.5 Soil principles for climate change adaptation and mitigation, and enhancing resilience  114
List of Tables
Table 5.1 Capacity of a soil with a bulk density of 1.2 g/cm3 to store water 
as affected by SOC content to 30 cm soil depth.  117
List of Boxes
Box 4.1 Carbon and SOM  117
Box 4.2 Inorganic fertil

2094

In [15]:
from dotenv import load_dotenv
# import os
from mistralai import Mistral

model = Mistral()

# load_dotenv()  # Load environment variables

# api_key = os.getenv("D:/Programs/Nirman_2025/Enviroment.env")
# db_user = os.getenv("DB_USER")
# db_pass = os.getenv("DB_PASS")

# print(api_key, db_user, db_pass)


In [54]:
'''
Query = "what type of soil is reqquired for crop"
results = collection.query(
    query_texts= Query, # Chroma will embed this for you
    n_results=20 # how many results to return
)

print(results)'''
Query  ="HOw to imporve soil fertility? "
query_embedding = res.encode([Query])
results = collection.query(embeddings[0])

In [20]:
'''from mistralai import Mistral
from langchain_mistralai.chat_models import ChatMistralAI
from langchain import VectorDBQA, PromptTemplate

geneartor = Mistral()

template = PromptTemplate(input_variables=["question"],
    template="Answer the following question: {Query}")
#qa_pipeline =  VectorDBQA(vector_store=collection, embeddings_model = res , prompt_template = template, language_model = geneartor)
response = qa_pipeline(Query)

print(response)'''
import os
from langchain_mistralai.chat_models import ChatMistralAI

# Set your Mistral AI API key (Replace "your-api-key" with your actual key)
os.environ["MISTRAL_API_KEY"] = "236mWUjffs24Rg2pkQNfQiJNxg9EUxNO"

def generate_response(prompt: str, model_name: str = "mistral-small", temperature: float = 0.7) -> str:
    """
    Generate a response using the Mistral AI model.

    Args:
        prompt (str): The input question or prompt.
        model_name (str): The Mistral model to use (default: "mistral-small").
        temperature (float): The temperature setting for creativity (default: 0.7).

    Returns:
        str: The generated response from Mistral AI.
    """
    try:
        # Initialize Mistral AI model with API key
        generator = ChatMistralAI(
            model=model_name,  # Model name (e.g., "mistral-small")
            temperature=temperature,
            mistral_api_key=os.getenv("MISTRAL_API_KEY")  # Fetch API key from env variable
        )

        # Generate and return the response
        response = generator.predict(prompt)
        return response
    except Exception as e:
        return f"Error: {e}"

# Example usage
# if __name__ == "__main__":
#     user_prompt = "How to imporve fertility in soil"
#     response = generate_response(prompt=user_prompt)
#     print("Generated Response:", response)


In [23]:
# Example: Input prompt
prompt = "which crops to grow in low moisture and harsh winter season?"

# Call the function
response = generate_response(prompt=prompt)

# Display the response
print("Generated Response:", response)


Generated Response: In a low moisture and harsh winter season, it is essential to choose crops that are drought-tolerant and can withstand cold temperatures. Here are some crops that you might consider growing:

1. Winter Wheat: Winter wheat is a variety of wheat that is planted in the fall and harvested in the spring. It is more cold-tolerant than spring wheat and can survive in low moisture conditions.
2. Barley: Barley is a hardy crop that can tolerate cold temperatures and low moisture levels. It is often used as a cover crop to prevent erosion and improve soil health.
3. Oats: Oats are another crop that can tolerate cold temperatures and low moisture levels. They are often used as a cover crop or for livestock feed.
4. Alfalfa: Alfalfa is a perennial legume that is drought-tolerant and can survive in cold temperatures. It is often used as a hay crop or for livestock feed.
5. Winter Rye: Winter rye is a cereal grain that is planted in the fall and harvested in the spring. It is mor